In [1]:
import pandas as pd
import numpy as np
import pathlib
import datetime
from rich import print
import xarray as xr

In [2]:
pth = pathlib.Path.home() / pathlib.Path("als/cvsbdata", follow_links=True)

In [3]:
pth.resolve()

PosixPath('/Users/kdavis10/als/cvsbdata')

In [4]:
try:
    del cl
except NameError:
    pass

cl = pd.read_csv("/Users/kdavis10/iCloud_Documents/Documents/Personal/Records/csvb/data/Combined Ledger.csv",
                    header=3, parse_dates=["Date"])

In [5]:
cl["Amount"] = cl["Amount"].astype(float)

In [6]:
xcl = cl.set_index("LedgerID").to_xarray()

In [7]:
# Add a dummmy dimension, will make transposing easier later.
xcl = xcl.expand_dims("dummy")

# Transfers from one account to another would have opposite signs on the amount.
amount_matches = xcl.Amount.data == -1 * xcl.Amount.data.T

# Dates should be within 5 days.
date_matches = np.abs(xcl.Date.data - xcl.Date.data.T).astype('timedelta64[D]')  < np.timedelta64(5, "D")

# Apply both for list.
matchix = np.where(np.logical_and(amount_matches, date_matches))

# Zip to tuples.
pairs = zip(matchix[0], matchix[1])

# Since (i, j) = (j, i) sort all pairs so that they can be compared, drop items that matched themselves, and use a set to drop duplicates.
pairs = list(set([tuple(np.sort(pair)) for pair in pairs if pair[0] != pair[1]]))

# Get LedgerIDs
#xfer_LIDs = (xcl.isel(LedgerID=xfer_matches[0])["LedgerID"], 
#                 xcl.isel(LedgerID=xfer_matches[1])["LedgerID"])


matcharray = np.array(pairs).T

# Remove dummy dimension.
xcl = xcl.squeeze()

In [8]:
#left = xcl.isel(LedgerID=matcharray[0,:])
#right = xcl.isel(LedgerID=matcharray[1,:])

#data_vars = {"matches": (("matchID", "pair"),
#                         matcharray)
#            }

#coords = {"matches": (("matchID", np.arange(matcharray.shape[0]),
#                       "pair", ["left", "right"])
#                     )
#         }

# Create a dataarray to old the matches, to be added to xcl.
pairda = xr.DataArray(data=matcharray,
                   coords={"match": ["left", "right"],
                           "matchID": np.arange(matcharray.shape[1]),
                          }
                     )
# Convert match indices to LedgerIDs and add to xcl.
matchLIDS = xcl.LedgerID.isel(LedgerID=pairda)
xcl["matches"]=matchLIDS
                      
#left = left.where(left.Amount!=0)
#right = xcl.isel(LedgerID=pairarray[:,1]).sel(Amount=0, drop=True)
#matches = (left, right)

In [9]:
# Create an array of nans to hold confirmation information.

xcl["confirmed_match"] = xr.DataArray(data=np.full((len(xcl.matchID),),np.NaN),
                                     dims=["matchID"],
                                     attrs={"Note": f"{np.nan} means not reviewed"})

In [10]:
xcl

<xarray.Dataset>
Dimensions:          (LedgerID: 1540, match: 2, matchID: 66)
Coordinates:
  * LedgerID         (LedgerID) object '7B2F4917' '051E10F3' ... '173FDAAB'
  * match            (match) <U5 'left' 'right'
  * matchID          (matchID) int64 0 1 2 3 4 5 6 7 ... 58 59 60 61 62 63 64 65
Data variables:
    Ledger           (LedgerID) object '1st Source:Checking' ... 'Fidelity:HSA'
    Date             (LedgerID) datetime64[ns] 2021-12-31 ... 2023-06-02
    Description      (LedgerID) object 'Initial Balance' ... ' PARTIC CONTR C...
    Amount           (LedgerID) float64 5.068e+03 -674.3 2e+03 ... -279.2 279.2
    Account          (LedgerID) object 'Initial Balance' ... 'Paycheck:Pretax'
    matches          (match, matchID) object 'E5013702' ... '3EE2D058'
    confirmed_match  (matchID) float64 nan nan nan nan nan ... nan nan nan nan

In [11]:
xcl.confirmed_match

<xarray.DataArray 'confirmed_match' (matchID: 66)>
array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan])
Coordinates:
  * matchID  (matchID) int64 0 1 2 3 4 5 6 7 8 9 ... 57 58 59 60 61 62 63 64 65
Attributes:
    Note:     nan means not reviewed

In [27]:
def match_to_str(match):
    s = f"Ledger: {str(match['pair' == 'left'].data)} -- {match['pair' == 'right'].data}"
    return s

In [67]:
def get_match(ds, matchID):
    m = xr.DataArray(ds.sel(LedgerID = ds.matches.sel(matchID=matchID).data))
    m.attrs["name"] = "match"
    
    

In [68]:
get_match(xcl, 0)

TypeError: cannot directly convert an xarray.Dataset into a numpy array. Instead, create an xarray.DataArray first, either with indexing on the Dataset or by invoking the `to_dataarray()` method.

In [100]:
ds = xcl
matchID=0
m = ds.sel(LedgerID = ds.matches.sel(matchID=matchID).data).drop_dims(["match", "matchID"])

In [102]:
m.attrs["name"] = "match"

In [110]:
m["match"] = ["left", "right"]